In [ ]:
import os
import numpy as np

from pathlib import Path

import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community.quality import modularity


output_file = "ideagraph.dot"
input_file = "CD_input.txt"


## preset values
# r  -> top 30 high-frequency items
# Nc = 12
M1 = 12
M2 = 12

# read files
txt_file = open(input_file, 'r')
lines = txt_file.readlines()
txt_file.close()


# -------------------------------- ## Step 1 A ## ----------------------------------
# compute the number of item
# compute the number of item-item pairs

# item/topic frequency list in asc order
items_list = []
for line in lines:
    items = line.split(" ")[:-1]
    items = [item for item in items]
    items_list.append(items)

#print('items_list:', items_list[:20])


tf = {}
for items in items_list:
    for item in items:
        if item in tf:
            tf[item] += 1
        else:
            tf[item] = 1
#print('tf:',tf)


# pair frequency
pf = {}         
for items in items_list:
    for i in items:
        for j in items:
            if i not in pf:
                pf[i] = {}
            if j not in pf[i]:
                pf[i][j] = 1
            else:
                pf[i][j] += 1

#print('pf:',pf)


#pf_list = []
#for x in pf.keys():
#    for y in pf[x].keys():
#        pf_list.append([x, y, pf[x][y]])

#pf_list = pf_list.sort(key=lambda a: a[2])

#print(pf_list)



# -----------------------------------## Step 1 B ## -------------------------------------

# compute the value of all clusters C and G 

R = {}
for i in tf.keys():
    for j in tf.keys():
        if i not in R:
            R[i] = {}
        assert j not in R[i]
        if j not in pf[i]:
            pf[i][j] = 0
        if i not in pf[j]:
            pf[j][i] = 0
        assert pf[i][j] == pf[j][i]
        R[i][j] = pf[i][j] / tf[j] + pf[j][i] / tf[i]
            

#print(len(R))
#print('R:', R)

#-------- ## compute preset value - r  ## ----------------

#get list of  R value, sorted in desc order
l = []
for i in pf.keys():
    for j in pf.keys():
        x = R.get(i, {}).get(j)
        if x != 2:
            l.append(x)
#print(l)


# match KeyGraph default preset value - link 30 pairs with black line
sorted_l = sorted(l, reverse=True)
#print('sorted_l:', sorted_l)
r = sorted_l[M]


#-------------- ## compute general clusters C, ## record gray point and gray line  ## ---------------------

C = []

gray_set = set()
gray_line = set()

for i in tf.keys():
    for j in tf.keys():
        if i != j and R[i][j] > r :
            ## find nodes in G
            gray_set.add(i)
            gray_set.add(j)
            if (i, j) not in gray_line and (j, i) not in gray_line:
                gray_line.add((i, j))
            
            ## emerge clusters
            new_c = True
            for C_i in C:
                if i in C_i or j in C_i:
                    C_i.add(i)
                    C_i.add(j)
                    new_c = False

            if new_c:
                C.append(set([i, j]))

G  = [i for i in C_i for C_i in C]
G  = list(set(G))

#print('gray_line', gray_line)
#print('gray_set', gray_set)
                
# Detect communities in the base and remove edges between clusters
def find_clusters(base):
    G = nx.Graph()
    for i, j in base:
        G.add_edge(i, j)
        
    communities = girvan_newman(G)
    communities_by_quality = [(c, modularity(G, c)) for c in communities]
    c_best = sorted([(c, m) for c, m in communities_by_quality], key=lambda x: x[1], reverse=True)
    c_best = c_best[0][0]
    # print(Util.pp(communities_by_quality))
    print("clusters:", modularity(G, c_best), c_best)
        
    # only include clusters of more than one node (for now)
    clusters = [c for c in c_best if len(c) > 1]

    # remove edges
    new_base = [edge for cluster in c_best for edge in G.subgraph(cluster).edges()]
        
    return clusters


C = find_clusters(gray_line)

#gray_set  = [i for i in C_i for C_i in C]
#gray_set  = list(set(gray_set))

#print('GC:',C)
#print('G:',G)



# -------------## Step 2，obtain cognitive clusters - top-Nc GC is CC  ## --------
infoC = []
#infoDenC = []
clusterValC = []
L = []
for k, C_i in enumerate(C):
    sum = 0
    for i in C_i:
        for j in C_i :
            sum += R[i][j]
    infoC.append((C_i, sum))
    #infoDenC.append((C_i, sum/(len(C_i))))
    clusterValC.append((C_i, 2 * sum / (len(C_i) + 1)))
    L.append(2 * sum / (len(C_i) + 1))
    
#print('infoC:', infoC)


## descending order
#sorted_infoDenC = sorted(infoDenC, key=lambda x:x[1], reverse=True)
sorted_clusterValC = sorted(clusterValC, key=lambda x:x[1], reverse=True)

#print('sorted_infoDenC:', sorted_infoDenC)
#print('sorted_clusterValC:', sorted_clusterValC)

### ---------------------------------test Nc ---------------------------
Nc = np.mean(L)
#print('Nc:', Nc)

## select top Nc as CC - cognitive clusters
CC = []
for C_i in range(min(len(sorted_clusterValC), round(Nc))):
    CC.append(sorted_clusterValC[C_i][0])

#print('CC:', CC)




# -----------------## # Step 3，Capture valuable links, top M1 #-------------------
## compute top M1 pairs
PR = []
for i in tf.keys():
    for CC_i_index, CC_i in enumerate(CC):
        if i not in CC_i:
            PR_value = 0
            for ck in CC_i:
                PR_value += R[i][ck]
            PR.append((i, CC_i_index, PR_value))

## sorting item-cluster pairs
sorted_PR = sorted(PR, key=lambda x:x[0], reverse=True)

#print('sorted_PR:',sorted_PR)

## selecting top M1 pairs

blue_set = set()
blue_line = set()

for k in range(min(len(sorted_PR), M1)):
    i = sorted_PR[k][0]
    CC_i = CC[sorted_PR[k][1]]

    # record blue line
    max_R = -1
    max_R_ck = None
    for ck in CC_i:
        if R[i][ck] > max_R:
            max_R = R[i][ck]
            max_R_ck = ck
    if (i, max_R_ck) not in blue_line and (max_R_ck, i) not in blue_line and (i, max_R_ck) not in gray_line and (max_R_ck, i) not in gray_line :
        blue_line.add((i, max_R_ck)) 

    # if i is  NOT in G, then add i as bule node
    if i not in gray_set and i not in blue_set:
        blue_set.add(i)

#print('Valuable links:', blue_line)
#print('blue_set:',blue_set)


# ----------------- # Step 4，Extracting key items - top M2 # ----------------- # 
## compute Key value

CC_all = [ item for elem in CC for item in elem]
CC_all = list(set(CC_all))
#print('CC_all:', CC_all)


Key = {}
Key_list = []

for i in tf.keys():
    PR_sum = 0
    for CC_i_index, CC_i in enumerate(CC):
        PR_value = 0
        for ck in CC_i:
            PR_value += R[i][ck]
        PR_sum += PR_value

    R_sum = 0
    for ik in (set(G)- set(CC_all)):
        R_sum += R[i][ik]

    Key[i] = PR_sum + R_sum
    Key_list.append((i, Key[i]))

sorted_Key_list = sorted(Key_list, key=lambda x:x[0], reverse=True)

#print('sorted_Key_list :', sorted_Key_list)


red_set = set()
red_line = set()

for k in range(min(len(sorted_Key_list), M2)):
    i = sorted_Key_list[k][0]
   # CC_i = CC[sorted_Key_list[k][1]]
    
    # record red point (newly added items)
    if i not in gray_set and i not in blue_set and i not in red_set:
        red_set.add(i)
        # record red line : red nodes -> CC
    if i in red_set:
        max_R = -1
        max_R_ck = None
        for CC_i_index, CC_i in enumerate(CC):
            for ck in CC_i:
                if R[i][ck] > max_R:
                    max_R = R[i][ck]
                    max_R_ck = ck
        if i != max_R_ck and (i, max_R_ck) not in gray_line and (i, max_R_ck) not in blue_line and (i, max_R_ck) not in red_line and (max_R_ck, i) not in gray_line  and (max_R_ck, i) not in blue_line and (max_R_ck, i) not in red_line:
            red_line.add((i, max_R_ck))

#print('Top M2 key items:', red_set)
#print('red_line:', red_line)         
    

# print(gray_set, blue_set, red_set, gray_line, red_line)
## create dot file
s = ""
s += "graph ideagraph {\ngraph [size=\"10,10\", overlap=\"scale\"]; node [fontsize=24];\n"
for item in gray_set:
    s += "%s [color=\"gray\"]\n" % ("\"" + item + "\"")
for item in blue_set:
    s += "%s [color=\"blue\"]\n" % ("\"" + item + "\"")
for item in red_set:
    s += "%s [color=\"red\"]\n" % ("\"" + item + "\"")
for line in gray_line:
    s += "%s--%s\n" % ("\"" + line[0] + "\"", "\"" + line[1] + "\"")
for line in blue_line:
    s += "%s--%s [color=\"blue\", style=\"dotted\"]\n" % ("\"" + line[0] + "\"", "\"" + line[1] + "\"")
for line in red_line:
    s += "%s--%s [color=\"red\", style=\"dotted\"]\n" % ("\"" + line[0] + "\"", "\"" + line[1] + "\"")
    
s += "}"

with open(output_file, 'w') as file:
    file.write(s)
    
print(s)



clusters: 0.8199999999999997 ({'T33', 'T51'}, {'T56', 'T35'}, {'T11', 'T3'}, {'T54', 'T45', 'T8'}, {'T24', 'T44', 'T17', 'T55'}, {'T0', 'T32'}, {'T59', 'T29'}, {'T34', 'T21'}, {'T41', 'T7'}, {'T57', 'T46'}, {'T58', 'T14'})
graph ideagraph {
graph [size="10,10", overlap="scale"]; node [fontsize=24];
"T46" [color="gray"]
"T55" [color="gray"]
"T21" [color="gray"]
"T33" [color="gray"]
"T7" [color="gray"]
"T17" [color="gray"]
"T35" [color="gray"]
"T51" [color="gray"]
"T45" [color="gray"]
"T29" [color="gray"]
"T0" [color="gray"]
"T34" [color="gray"]
"T41" [color="gray"]
"T54" [color="gray"]
"T24" [color="gray"]
"T56" [color="gray"]
"T11" [color="gray"]
"T58" [color="gray"]
"T44" [color="gray"]
"T57" [color="gray"]
"T8" [color="gray"]
"T32" [color="gray"]
"T14" [color="gray"]
"T3" [color="gray"]
"T59" [color="gray"]
"T6" [color="blue"]
"T9" [color="blue"]
"T53" [color="red"]
"T52" [color="red"]
"T33"--"T51"
"T56"--"T35"
"T3"--"T45"
"T44"--"T24"
"T55"--"T17"
"T32"--"T0"
"T29"--"T59"
"T8"--"T45